In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#to ignore warnings
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)

from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score


In [ ]:
import openai

# Set your API Key
openai.api_key = "your-api-key-here"

def chat_with_gpt(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Use "gpt-3.5-turbo" if needed
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

# Test the connection
user_input = "Explain the concept of reinforcement learning."
print(chat_with_gpt(user_input))

In [17]:
import os
os.chdir('D:\Learning and Research\Data science\Kaggle\Competitions\Car price prediction')

In [18]:
cprice_train = pd.read_csv('train.csv')
cprice_test = pd.read_csv('test.csv')
cprice_train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [ ]:
cprice_train.describe(include='all').T

In [ ]:
cprice_train.isnull().sum()/(len(cprice_train))

In [ ]:
cat_cols = cprice_train.select_dtypes(include=['object']).columns
num_cols = cprice_train.loc[:, cprice_train.columns != 'id'].select_dtypes(include=np.number).columns.tolist()
print("Categorical Variables:")
print(cat_cols)
print("Numerical Variables:")
print(num_cols)

In [ ]:
cprice_train[cprice_train['fuel_type'].isnull()].groupby('brand', dropna=False).count()/cprice_train.groupby('brand', dropna=False).count()

In [20]:
cprice_train.groupby(['fuel_type','engine'],dropna = False).agg({'id': 'count'}).sort_values('id', ascending=False)

id
fuel_type      engine                                                  
Gasoline       355.0HP 5.3L 8 Cylinder Engine Gasoline Fuel        3460
               240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel        2899
               420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel        2841
               2.0L I4 16V GDI DOHC Turbo                          2673
               375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel       2448
               340.0HP 3.0L V6 Cylinder Engine Gasoline Fuel       2435
               490.0HP 6.2L 8 Cylinder Engine Gasoline Fuel        2184
               455.0HP 6.2L 8 Cylinder Engine Gasoline Fuel        2138
               425.0HP 3.0L Straight 6 Cylinder Engine Gasolin...  2133
               300.0HP 3.0L Straight 6 Cylinder Engine Gasolin...  2084
               295.0HP 3.5L V6 Cylinder Engine Gasoline Fuel       1916
               285.0HP 3.6L V6 Cylinder Engine Gasoline Fuel       1908
               302.0HP 3.5L V6 Cylinder Engine Gasoline Fuel       1894
               3.5L V6 24V PDI DOHC Twin Turbo                     1839
               301.0HP 4.6L 8 Cylinder Engine Gasoline Fuel        1578
               5.7L V8 16V MPFI OHV                                1518
               4.0L V8 32V GDI DOHC Twin Turbo                     1505
               3.6L V6 24V MPFI DOHC                               1476
               268.0HP 3.5L V6 Cylinder Engine Gasoline Fuel       1470
               333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel       1318
               365.0HP 3.5L V6 Cylinder Engine Gasoline Fuel       1274
               211.0HP 2.0L 4 Cylinder Engine Gasoline Fuel        1234
               3.6L V6 24V GDI DOHC                                1221
               210.0HP 3.7L V6 Cylinder Engine Gasoline Fuel       1132
               252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel        1119
               2.0 Liter Turbo                                     1118
               248.0HP 2.0L 4 Cylinder Engine Gasoline Fuel        1080
               255.0HP 2.0L 4 Cylinder Engine Gasoline Fuel        1054
               380.0HP 3.0L V6 Cylinder Engine Gasoline Fuel       1026
               5.3L V8 16V GDI OHV                                 1008
               381.0HP 5.7L 8 Cylinder Engine Gasoline Fuel        1008
               395.0HP 5.7L 8 Cylinder Engine Gasoline Fuel         980
Hybrid         335.0HP 3.0L V6 Cylinder Engine Gasoline/Mild E...   963
Gasoline       362.0HP 3.0L V6 Cylinder Engine Gasoline Fuel        955
               290.0HP 3.5L V6 Cylinder Engine Gasoline Fuel        949
               241.0HP 2.0L 4 Cylinder Engine Gasoline Fuel         925
               325.0HP 2.7L V6 Cylinder Engine Gasoline Fuel        901
               2.5L I4 16V GDI DOHC                                 896
               445.0HP 4.4L 8 Cylinder Engine Gasoline Fuel         883
               306.0HP 3.5L V6 Cylinder Engine Gasoline Fuel        843
               414.0HP 4.0L 8 Cylinder Engine Gasoline Fuel         836
               400.0HP 5.6L 8 Cylinder Engine Gasoline Fuel         830
Hybrid         395.0HP 3.0L Straight 6 Cylinder Engine Gasolin...   828
Gasoline       3.5L V6 24V PDI DOHC                                 813
               450.0HP 3.5L V6 Cylinder Engine Gasoline Fuel        809
               560.0HP 4.4L 8 Cylinder Engine Gasoline Fuel         808
               503.0HP 3.0L Straight 6 Cylinder Engine Gasolin...   785
E85 Flex Fuel  320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capabi...   781
–              –                                                    772
Gasoline       270.0HP 4.0L V6 Cylinder Engine Gasoline Fuel        764
NaN            518.0HP Electric Motor Electric Fuel System          753
Gasoline       270.0HP 3.5L V6 Cylinder Engine Gasoline Fuel        743
E85 Flex Fuel  385.0HP 6.2L 8 Cylinder Engine Flex Fuel Capabi...   732
Gasoline       6.2L V8 16V GDI OHV                                  725
               416.0HP 4.0L 8 Cylinde

In [ ]:
def segregate_engine(df):
    # Split the engine column into multiple columns
    engine_split = df['engine'].str.split(' ', expand=True)
    
    # Create a new column for the cylinder information
    df['cylinder_info'] = engine_split[engine_split.apply(lambda x: 'Cylinder' in x, axis=1)].apply(lambda x: ' '.join(x.dropna()), axis=1)
    
    # Drop the original engine column
    df.drop(columns=['engine'], inplace=True)
    
    # Concatenate the new columns to the original dataframe
    df = pd.concat([df, engine_split], axis=1)
    
    return df

# Apply the function to the dataframe
cprice_train = segregate_engine(cprice_train)
cprice_train.head(100)

In [ ]:
cprice_train.groupby(['0','1','2','3','4','5','6','7','8','9'],dropna = False).agg({'id': 'count'}).sort_values('id', ascending=False)